Before starting we manually split the images into to sub-images that conctained 1 leaf per image. As before every image could contain from 1 to 6 leaves. After we implemented a DNN algorithm U^2-net that removed the backround from each image and produced a .png file without the backround noise that would insert bias into our NNs.

In [ ]:
## Backround removal procedure ##

images_dir = '/content/Test_images'
if os.path.exists(images_dir):
    shutil.rmtree(images_dir)
    os.makedirs(images_dir)
    print(f'Directory {images_dir} has been emptied and recreated.')
else:
    print(f'Directory {images_dir} does not exist. Creating it now.')
    os.makedirs(images_dir)


%cd /content
!/usr/local/cuda/bin/nvcc --version
!git clone https://github.com/shreyas-bk/U-2-Net
%cd /content/U-2-Net
print('making images directory')
!mkdir images
print('making results directory')
!mkdir results
print('importing...')
from google.colab import files
import os
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array
import numpy as np
from PIL import Image as Img
import cv2
from google.colab.patches import cv2_imshow
print('Done!')

%cd /content/Test_images
uploaded = files.upload()
%cd /content/U-2-Net


!python -W ignore u2net_test.py

image_dir = os.path.join(os.getcwd(), 'images')
names = [name[:-4] for name in os.listdir(image_dir)]
THRESHOLD = 0.9
RESCALE = 255
LAYER = 2
COLOR = (0, 0, 0)
THICKNESS = 4
SAL_SHIFT = 100


for name in names:


  if name == '.ipynb_checkpo':
    continue
  output = load_img('/content/U-2-Net/results/'+name+'.png')
  out_img = img_to_array(output)
  out_img /= RESCALE

  out_img[out_img > THRESHOLD] = 1
  out_img[out_img <= THRESHOLD] = 0

  shape = out_img.shape
  a_layer_init = np.ones(shape = (shape[0],shape[1],1))
  mul_layer = np.expand_dims(out_img[:,:,0],axis=2)
  a_layer = mul_layer*a_layer_init
  rgba_out = np.append(out_img,a_layer,axis=2)

  input = load_img('/content/Test_images/'+name+'.jpg')
  inp_img = img_to_array(input)
  inp_img /= RESCALE

  a_layer = np.ones(shape = (shape[0],shape[1],1))
  rgba_inp = np.append(inp_img,a_layer,axis=2)

  rem_back = (rgba_inp*rgba_out)
  rem_back_scaled = rem_back*RESCALE

  inp_img*=RESCALE
  inp_img = np.append(inp_img,RESCALE*a_layer,axis=2)
  inp_img = cv2.resize(inp_img,(int(shape[1]/3),int(shape[0]/3)))
  rem_back = cv2.resize(rem_back_scaled,(int(shape[1]/3),int(shape[0]/3)))
  result = np.concatenate((inp_img,rem_back),axis=1)
  result_img = Img.fromarray(result.astype('uint8'), 'RGBA')
  print('\nINPUT                                    BACKGROUND REMOVED')
  display(result_img)

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
## Reading the raw file and dropping the uneeded values ##

SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
col_names = list(SSR_data.columns)
col_names_1 = col_names[::2]
col_names_2 = col_names[1::2]

In [ ]:
## Generating list with unique alles in each genetic locus ##

all_js = []
Uniques = []
for name1, name2 in zip(col_names_1, col_names_2):
    y = SSR_data[name1]
    i = list(y.unique())
    z = SSR_data[name2]
    j = list(z.unique())

    for k in range(len(i)):
      if (i[k] not in j) and (pd.isna(i[k])== False):
        j.append(i[k])
    print(j)
    Uniques = Uniques + j
    np.array(j)
    all_js.append(j)

In [ ]:
## Plot the species distribution to our raw/non-augmented data ##

import matplotlib.pyplot as plt


data = {'Petrea':36, 'Fraineto':15, 'Pubescens':7}
species = list(data.keys())
count = list(data.values())

fig = plt.figure(figsize = (10, 5))

plt.bar(species, count, color ='maroon',
        width = 0.4)

plt.xlabel("Species")
plt.ylabel("No. of trees in each species")
plt.show()

In [ ]:
## Creating a file to see the unique alles combinations ##

file = open("unique_combs.txt", 'w')

for name in concatenated_data_col_names_B:
  y = final_binary[name]
  i = list(y.unique())

  print(name)
  print(i)
  print('\n')
  print(len(i))
  print('\n')

    file.write(name)
  file.write('\n')
  file.write('\n')
  file.write(str(i))
  file.write('\n')
  file.write(str(len(i)))
  file.write('\n')
  file.write('\n')
  file.write('\n')


file.close()

Creating the one-hot encoding and doing it from every allele in each locus

In [ ]:
## One-hot encoding for the alleles in Pie215 locus ##

SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [218, 215, 194, 203, 209, 206, 212, 200, 197, 227, 188]

for i in range(len(l)):
  for j in range(len(SSR_data["Pie215"])):
    if SSR_data["Pie215"][j] == l[i]:
      SSR_data["Pie215"][j] = 1
    else:
      SSR_data["Pie215"][j] = 0
  for j in range(len(SSR_data["Pie215.1"])):
    if SSR_data["Pie215.1"][j] == l[i]:
      SSR_data["Pie215.1"][j] = 1
    else:
      SSR_data["Pie215.1"][j] = 0


  col_name = "Pie215-" + str(l[i])
  col_name_1 = "Pie215.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["Pie215"]
  df.loc[:, col_name_1] = SSR_data["Pie215.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [79, 98, 84, 88, 85, 83, 81, 86, 77, 82]

for i in range(len(l)):
  for j in range(len(SSR_data["Pie223"])):
    if SSR_data["Pie223"][j] == l[i]:
      SSR_data["Pie223"][j] = 1
    else:
      SSR_data["Pie223"][j] = 0
  for j in range(len(SSR_data["Pie223.1"])):
    if SSR_data["Pie223.1"][j] == l[i]:
      SSR_data["Pie223.1"][j] = 1
    else:
      SSR_data["Pie223.1"][j] = 0


  col_name = "Pie223-" + str(l[i])
  col_name_1 = "Pie223.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["Pie223"]
  df.loc[:, col_name_1] = SSR_data["Pie223.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [217, 199, 211, 205, 220, 214, 233, 223, 208]

for i in range(len(l)):
  for j in range(len(SSR_data["Pie227"])):
    if SSR_data["Pie227"][j] == l[i]:
      SSR_data["Pie227"][j] = 1
    else:
      SSR_data["Pie227"][j] = 0
  for j in range(len(SSR_data["Pie227.1"])):
    if SSR_data["Pie227.1"][j] == l[i]:
      SSR_data["Pie227.1"][j] = 1
    else:
      SSR_data["Pie227.1"][j] = 0


  col_name = "Pie227-" + str(l[i])
  col_name_1 = "Pie227.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["Pie227"]
  df.loc[:, col_name_1] = SSR_data["Pie227.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [161, 174, 155, 171, 158, 164, 148]

for i in range(len(l)):
  for j in range(len(SSR_data["Pie239"])):
    if SSR_data["Pie239"][j] == l[i]:
      SSR_data["Pie239"][j] = 1
    else:
      SSR_data["Pie239"][j] = 0
  for j in range(len(SSR_data["Pie239.1"])):
    if SSR_data["Pie239.1"][j] == l[i]:
      SSR_data["Pie239.1"][j] = 1
    else:
      SSR_data["Pie239.1"][j] = 0


  col_name = "Pie239-" + str(l[i])
  col_name_1 = "Pie239.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["Pie239"]
  df.loc[:, col_name_1] = SSR_data["Pie239.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [171, 170, 179, 177, 160, 173, 166, 162, 164, 158, 149, 153, 140, 155, 147, 136, 144, 151]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG96"])):
    if SSR_data["ZAG96"][j] == l[i]:
      SSR_data["ZAG96"][j] = 1
    else:
      SSR_data["ZAG96"][j] = 0
  for j in range(len(SSR_data["ZAG96.1"])):
    if SSR_data["ZAG96.1"][j] == l[i]:
      SSR_data["ZAG96.1"][j] = 1
    else:
      SSR_data["ZAG96.1"][j] = 0


  col_name = "ZAG96-" + str(l[i])
  col_name_1 = "ZAG96.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG96"]
  df.loc[:, col_name_1] = SSR_data["ZAG96.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [96, 85, 87, 94, 106, 79, 81, 83]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG112"])):
    if SSR_data["ZAG112"][j] == l[i]:
      SSR_data["ZAG112"][j] = 1
    else:
      SSR_data["ZAG112"][j] = 0
  for j in range(len(SSR_data["ZAG112.1"])):
    if SSR_data["ZAG112.1"][j] == l[i]:
      SSR_data["ZAG112.1"][j] = 1
    else:
      SSR_data["ZAG112.1"][j] = 0


  col_name = "ZAG112-" + str(l[i])
  col_name_1 = "ZAG112.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG112"]
  df.loc[:, col_name_1] = SSR_data["ZAG112.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [218.0, 226.0, 228.0, 232.0, 222.0, 235.0, 230.0, 224.0, nan, 220.0, 234.0, 196.0, 212.0, 216.0, 210.0, 200.0]

for i in range(len(l)):
  for j in range(len(SSR_data["MSQ13"])):
    if SSR_data["MSQ13"][j] == l[i]:
      SSR_data["MSQ13"][j] = 1
    else:
      SSR_data["MSQ13"][j] = 0
  for j in range(len(SSR_data["MSQ13.1"])):
    if SSR_data["MSQ13.1"][j] == l[i]:
      SSR_data["MSQ13.1"][j] = 1
    else:
      SSR_data["MSQ13.1"][j] = 0


  col_name = "MSQ13-" + str(l[i])
  col_name_1 = "MSQ13.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["MSQ13"]
  df.loc[:, col_name_1] = SSR_data["MSQ13.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [167.0, nan, 171.0, 193.0, 165.0, 173.0, 177.0, 175.0, 181.0, 169.0, 183.0, 187.0, 179.0, 161.0, 163.0, 157.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG20"])):
    if SSR_data["ZAG20"][j] == l[i]:
      SSR_data["ZAG20"][j] = 1
    else:
      SSR_data["ZAG20"][j] = 0
  for j in range(len(SSR_data["ZAG20.1"])):
    if SSR_data["ZAG20.1"][j] == l[i]:
      SSR_data["ZAG20.1"][j] = 1
    else:
      SSR_data["ZAG20.1"][j] = 0


  col_name = "ZAG20-" + str(l[i])
  col_name_1 = "ZAG20.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG20"]
  df.loc[:, col_name_1] = SSR_data["ZAG20.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [175.0, 172.0, nan, 186.0, 179.0, 170.0, 168.0, 173.0, 174.0, 167.0, 176.0, 184.0, 171.0, 164.0, 177.0, 169.0, 159.0, 166.0, 161.0, 160.0, 163.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG15"])):
    if SSR_data["ZAG15"][j] == l[i]:
      SSR_data["ZAG15"][j] = 1
    else:
      SSR_data["ZAG15"][j] = 0
  for j in range(len(SSR_data["ZAG15.1"])):
    if SSR_data["ZAG15.1"][j] == l[i]:
      SSR_data["ZAG15.1"][j] = 1
    else:
      SSR_data["ZAG15.1"][j] = 0


  col_name = "ZAG15-" + str(l[i])
  col_name_1 = "ZAG15.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG15"]
  df.loc[:, col_name_1] = SSR_data["ZAG15.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [252.0, 279.0, 271.0, 256.0, 275.0, 254.0, 248.0, 272.0, 267.0, 257.0, 243.0, 268.0, 264.0, nan, 250.0, 270.0, 240.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG11"])):
    if SSR_data["ZAG11"][j] == l[i]:
      SSR_data["ZAG11"][j] = 1
    else:
      SSR_data["ZAG11"][j] = 0
  for j in range(len(SSR_data["ZAG11.1"])):
    if SSR_data["ZAG11.1"][j] == l[i]:
      SSR_data["ZAG11.1"][j] = 1
    else:
      SSR_data["ZAG11.1"][j] = 0


  col_name = "ZAG11-" + str(l[i])
  col_name_1 = "ZAG11.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG11"]
  df.loc[:, col_name_1] = SSR_data["ZAG11.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [197, 193, 189, 199, 203, 211, 195, 207, 185, 187, 201, 191, 204, 247, 183, 181]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG9"])):
    if SSR_data["ZAG9"][j] == l[i]:
      SSR_data["ZAG9"][j] = 1
    else:
      SSR_data["ZAG9"][j] = 0
  for j in range(len(SSR_data["ZAG9.1"])):
    if SSR_data["ZAG9.1"][j] == l[i]:
      SSR_data["ZAG9.1"][j] = 1
    else:
      SSR_data["ZAG9.1"][j] = 0


  col_name = "ZAG9-" + str(l[i])
  col_name_1 = "ZAG9.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG9"]
  df.loc[:, col_name_1] = SSR_data["ZAG9.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [204.0, 222.0, 208.0, 234.0, 228.0, 224.0, 205.0, 218.0, 216.0, 220.0, 212.0, 210.0, 206.0, nan, 226.0, 202.0, 230.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG110"])):
    if SSR_data["ZAG110"][j] == l[i]:
      SSR_data["ZAG110"][j] = 1
    else:
      SSR_data["ZAG110"][j] = 0
  for j in range(len(SSR_data["ZAG110.1"])):
    if SSR_data["ZAG110.1"][j] == l[i]:
      SSR_data["ZAG110.1"][j] = 1
    else:
      SSR_data["ZAG110.1"][j] = 0


  col_name = "ZAG110-" + str(l[i])
  col_name_1 = "ZAG110.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG110"]
  df.loc[:, col_name_1] = SSR_data["ZAG110.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [115, 111, 110, 113]

for i in range(len(l)):
  for j in range(len(SSR_data["GOT021"])):
    if SSR_data["GOT021"][j] == l[i]:
      SSR_data["GOT021"][j] = 1
    else:
      SSR_data["GOT021"][j] = 0
  for j in range(len(SSR_data["GOT021.1"])):
    if SSR_data["GOT021.1"][j] == l[i]:
      SSR_data["GOT021.1"][j] = 1
    else:
      SSR_data["GOT021.1"][j] = 0


  col_name = "GOT021-" + str(l[i])
  col_name_1 = "GOT021.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["GOT021"]
  df.loc[:, col_name_1] = SSR_data["GOT021.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [156, 158, 154, 147, 143, 149, 145]

for i in range(len(l)):
  for j in range(len(SSR_data["GOT045"])):
    if SSR_data["GOT045"][j] == l[i]:
      SSR_data["GOT045"][j] = 1
    else:
      SSR_data["GOT045"][j] = 0
  for j in range(len(SSR_data["GOT045.1"])):
    if SSR_data["GOT045.1"][j] == l[i]:
      SSR_data["GOT045.1"][j] = 1
    else:
      SSR_data["GOT045.1"][j] = 0


  col_name = "GOT045-" + str(l[i])
  col_name_1 = "GOT045.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["GOT045"]
  df.loc[:, col_name_1] = SSR_data["GOT045.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [204, 198, 207, 212, 201, 209, 193, 195, 178]

for i in range(len(l)):
  for j in range(len(SSR_data["GOT062"])):
    if SSR_data["GOT062"][j] == l[i]:
      SSR_data["GOT062"][j] = 1
    else:
      SSR_data["GOT062"][j] = 0
  for j in range(len(SSR_data["GOT062.1"])):
    if SSR_data["GOT062.1"][j] == l[i]:
      SSR_data["GOT062.1"][j] = 1
    else:
      SSR_data["GOT062.1"][j] = 0


  col_name = "GOT062-" + str(l[i])
  col_name_1 = "GOT062.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["GOT062"]
  df.loc[:, col_name_1] = SSR_data["GOT062.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [218.0, nan, 225.0, 212.0, 215.0, 200.0, 216.0, 223.0, 209.0, 210.0, 202.0, 214.0, 226.0, 220.0, 222.0, 211.0, 208.0, 206.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG36"])):
    if SSR_data["ZAG36"][j] == l[i]:
      SSR_data["ZAG36"][j] = 1
    else:
      SSR_data["ZAG36"][j] = 0
  for j in range(len(SSR_data["ZAG36.1"])):
    if SSR_data["ZAG36.1"][j] == l[i]:
      SSR_data["ZAG36.1"][j] = 1
    else:
      SSR_data["ZAG36.1"][j] = 0


  col_name = "ZAG36-" + str(l[i])
  col_name_1 = "ZAG36.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG36"]
  df.loc[:, col_name_1] = SSR_data["ZAG36.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

In [ ]:
SSR_data = pd.read_excel('SSR_data.xlsx')
SSR_data = SSR_data.drop('Sample', axis=1)
SSR_data = SSR_data.drop('Species', axis=1)
SSR_data = SSR_data.drop('Longitude', axis=1)
SSR_data = SSR_data.drop('Latitude', axis=1)

l = [113.0, nan, 157.0, 135.0, 133.0, 131.0, 139.0, 123.0, 129.0, 127.0, 137.0, 115.0, 119.0, 125.0, 147.0, 121.0]

for i in range(len(l)):
  for j in range(len(SSR_data["ZAG7"])):
    if SSR_data["ZAG7"][j] == l[i]:
      SSR_data["ZAG7"][j] = 1
    else:
      SSR_data["ZAG7"][j] = 0
  for j in range(len(SSR_data["ZAG7.1"])):
    if SSR_data["ZAG7.1"][j] == l[i]:
      SSR_data["ZAG7.1"][j] = 1
    else:
      SSR_data["ZAG7.1"][j] = 0


  col_name = "ZAG7-" + str(l[i])
  col_name_1 = "ZAG7.1-" + str(l[i])
  df.loc[:, col_name] = SSR_data["ZAG7"]
  df.loc[:, col_name_1] = SSR_data["ZAG7.1"]


  SSR_data = pd.read_excel('SSR_data.xlsx')
  SSR_data = SSR_data.drop('Sample', axis=1)
  SSR_data = SSR_data.drop('Species', axis=1)
  SSR_data = SSR_data.drop('Longitude', axis=1)
  SSR_data = SSR_data.drop('Latitude', axis=1)

After creating the one-hot encoded file, we augment in and then we upload the augmented file that we will work on it from now on

In [ ]:
fin_oh = pd.read_excel('/content/Augmented_fin_fin_ohe.xlsx')
print(fin_oh.shape)
print(fin_oh.size)

In [ ]:
## Interger encoding the labels from petraea,frainetto,pubescens to 0,1,2 respectively to the augmented panda frame  ##

fin_oh['Species'] = fin_oh['Species'].map({'petraea':0,'frainetto':1,'pubescens':2})
species_labels = fin_oh['Species']

In [ ]:
## Loading the keras libraries ##

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
from imblearn.over_sampling import SMOTE

In [ ]:
## Running the NN to the non-augmented file ##


fin_oh = pd.read_excel('/content/fin_fin_ohe.xlsx')
fin_oh['Species'] = fin_oh['Species'].map({'petraea':0,'frainetto':1,'pubescens':2})
species_labels = fin_oh['Species']


data = fin_oh
X = data.drop('Species', axis=1).values
y = data['Species'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
learning_rate = 0.001


early_stopping = EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True)
model = Sequential()
model.add(Dense(12, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(3, activation='softmax'))

optimizer = Adam(learning_rate=learning_rate)
model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, batch_size=10, validation_split=0.3,callbacks=[early_stopping])

print(history.history)

y_pred = model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_labels)

print("Confusion Matrix:")
print(cm)

acc = history.history.get('accuracy', [])
val_acc = history.history.get('val_accuracy', [])
loss = history.history.get('loss', [])
val_loss = history.history.get('val_loss', [])

if len(acc) > 0 and len(val_acc) > 0:
    epochs = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, acc, label='Training Accuracy')
    plt.plot(epochs, val_acc, label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, loss, label='Training Loss')
    plt.plot(epochs, val_loss, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.tight_layout()
    plt.show()
else:
    print("No history data available for plotting.")

loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
## Creating the NN branch that will handle the augmented genetic data ##

data = fin_oh

X = data.drop('Species', axis=1).values
y = data['Species'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

learning_rate = 0.001

genetic_input = Input(shape=(X_train.shape[1],))
genetic_output = Dense(12, activation='swish')(genetic_input)
genetic_output = Dropout(0.5)(genetic_output)
genetic_output = Dense(8, activation='swish')(genetic_output)
genetic_output = Dense(3, activation='softmax')(genetic_output)

genetic_model = Model(inputs=genetic_input, outputs=genetic_output)

optimizer = Adam(learning_rate=learning_rate)
genetic_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

history = genetic_model.fit(X_train, y_train, epochs=20, batch_size=10, validation_split=0.3)

y_pred = genetic_model.predict(X_test)

y_pred_labels = np.argmax(y_pred, axis=1)

cm = confusion_matrix(y_test, y_pred_labels)

print("Confusion Matrix:")
print(cm)


In [ ]:
loss, accuracy = genetic_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy:.2f}')

In [ ]:
## Saving the model and the outputs ##

genetic_model.save('genetic_model.h5')
genetic_outputs = genetic_model.predict(X_test)
np.save('genetic_outputs.npy', genetic_outputs)

In [ ]:
## Loading the file containing the curated/backround removed pictures ##

from google.colab import files
uploaded = files.upload()

In [ ]:
## Some functions for the image branch, to delete directories concerning the images (This is done for code reruns as collab does not overwrite directories) ##

import shutil
import os

folder_path = '/content/preprocessed_images'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Successfully deleted the folder: {folder_path}")
else:
    print(f"The folder does not exist: {folder_path}")
    import shutil
import os

folder_path = '/content/Sorted'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Successfully deleted the folder: {folder_path}")
else:
    print(f"The folder does not exist: {folder_path}")

folder_path = '/content/Final_Curated_Data'

if os.path.exists(folder_path):
    shutil.rmtree(folder_path)
    print(f"Successfully deleted the folder: {folder_path}")
else:
    print(f"The folder does not exist: {folder_path}")

In [ ]:
## Some functions to handle and sort the images folder, interger encode them e.t.c ##
import shutil
import os
import random
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from tensorflow.keras.preprocessing import image
import cv2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50


petraea = list(range(1, 8)) + list(range(27,38)) + list(range(42,55)) + [9,23,25,58,59]
print(petraea)
frainetto = list(range(12,15)) + list(range(38,42)) + list(range(55,58)) + [8,19,20,22,24,26]
print(frainetto)
pubescens = list(range(15,19)) + [10,11,21]
print(pubescens)

class_names = ['Frainetto', 'Petraea', 'Pubescens']

input_folder = '/content/Sorted/'
train_folder = 'preprocessed_images/train'
test_folder = 'preprocessed_images/test'

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

source_dir = '/content/Final_Curated_Data/Final_Data_Resized'

dest_dirs = {
    'petraea': '/content/Sorted/Petraea',
    'frainetto': '/content/Sorted/Frainetto',
    'pubescens': '/content/Sorted/Pubescens'
}

for dir_path in dest_dirs.values():
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def move_images(numbers, dest_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith(".png"):
            parts = filename.split('_')[0]
            file_number = int(''.join(filter(str.isdigit, parts)))
            if file_number in numbers:
                source_path = os.path.join(source_dir, filename)
                dest_path = os.path.join(dest_dir, filename)
                shutil.move(source_path, dest_path)

move_images(petraea, dest_dirs['petraea'])
move_images(frainetto, dest_dirs['frainetto'])
move_images(pubescens, dest_dirs['pubescens'])


def load_npy_files(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.npy'):
            filepath = os.path.join(folder, filename)
            image = np.load(filepath)
            label_part = filename.split('_')[0]

            if label_part == 'Frainetto':
                label = 0
            elif label_part == 'Petraea':
                label = 1
            elif label_part == 'Pubescens':
                label = 2
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))
    img = np.array(img).astype(np.float32)
    img = preprocess_input(img)
    return img


image_filenames = []
for class_folder in class_names:
    class_path = os.path.join(input_folder, class_folder)
    for filename in os.listdir(class_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_filenames.append((class_folder, filename))

train_filenames, test_filenames = train_test_split(image_filenames, test_size=0.2, random_state=42)

for class_folder, filename in train_filenames:
    image_path = os.path.join(input_folder, class_folder, filename)
    preprocessed_image = preprocess_image(image_path)

    output_path = os.path.join(train_folder, f"{class_folder}_label_{class_names.index(class_folder)}_{os.path.splitext(filename)[0]}.npy")
    np.save(output_path, preprocessed_image)

for class_folder, filename in test_filenames:
    image_path = os.path.join(input_folder, class_folder, filename)
    preprocessed_image = preprocess_image(image_path)

    output_path = os.path.join(test_folder, f"{class_folder}_label_{class_names.index(class_folder)}_{os.path.splitext(filename)[0]}.npy")
    np.save(output_path, preprocessed_image)

train_images, train_labels = load_npy_files(train_folder)
test_images, test_labels = load_npy_files(test_folder)


print(f"Number of training images: {train_images.shape[0]}")
print(f"Number of test images: {test_images.shape[0]}")

print(f"Shape of train labels: {train_labels.shape}")
print(f"Shape of test labels: {test_labels.shape}")

In [ ]:
## Creating the NN for the image branch ##

num_classes = 3

resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet_base.trainable = False

image_input = Input(shape=(224, 224, 3))

x = resnet_base(image_input)

x = GlobalAveragePooling2D()(x)

x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)

image_output = Dense(num_classes, activation='softmax')(x)

image_model = Model(inputs=image_input, outputs=image_output)

image_model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = image_model.fit(train_images, train_labels, batch_size=32, epochs=10, validation_data=(test_images, test_labels))

test_loss, test_accuracy = image_model.evaluate(test_images, test_labels)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

pred_labels = image_model.predict(test_images)
pred_labels = np.argmax(pred_labels, axis=1)

true_labels = test_labels

conf_matrix = confusion_matrix(true_labels, pred_labels)
print("Confusion Matrix:")
print(conf_matrix)

report = classification_report(true_labels, pred_labels, target_names=class_names)
print(report)

In [ ]:
## Saving the image and results model ##

image_model.save('image_model.h5')
image_outputs = image_model.predict(test_images)
np.save('image_outputs.npy', image_outputs)

In [ ]:
## Creating the combined model ##

from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

merged = Concatenate()([image_output, genetic_output])

x = Dense(12, activation='swish')(merged)
x = Dense(8, activation='swish')(x)

output = Dense(num_classes, activation='softmax')(x)  # or 'sigmoid' for binary classification

combined_model = Model(inputs=[image_input, genetic_input], outputs=output)

optimizer = Adam(learning_rate=1e-2)
combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

history = combined_model.fit([train_images, X_train], y_train, batch_size=32, epochs=10, validation_split=0.2)

test_loss, test_accuracy = combined_model.evaluate([test_images, X_test], y_test)
print(f"Combined Test accuracy: {test_accuracy * 100:.2f}%")

In [ ]:
## Saving the combined model ##

combined_model.save('combined_model.h5')

Running the models over a 10 iteration loop to examine how they behave and generate the ribbon plots


In [ ]:

## 10 iterations of the genetic NN ##

import numpy as np
import joblib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

data = fin_oh

X = data.drop('Species', axis=1).values
y = data['Species'].values

num_runs = 10
epochs = 10

val_accuracy_each_run = []
test_accuracy_each_run = []
val_loss_each_run = []
test_loss_each_run = []

for run in range(num_runs):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42 + run, stratify=y)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    genetic_input = Input(shape=(X_train.shape[1],))
    genetic_output = Dense(12, activation='swish')(genetic_input)
    genetic_output = Dropout(0.5)(genetic_output)
    genetic_output = Dense(8, activation='swish')(genetic_output)
    genetic_output = Dense(3, activation='softmax')(genetic_output)
    genetic_model = Model(inputs=genetic_input, outputs=genetic_output)

    optimizer = Adam(learning_rate=0.001)
    genetic_model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    val_accuracy_current_run = []
    val_loss_current_run = []
    test_accuracy_current_run = []
    test_loss_current_run = []

    for epoch in range(epochs):
        history = genetic_model.fit(
            X_train, y_train,
            epochs=1,
            batch_size=10,
            validation_split=0.3,
            verbose=0
        )

        val_accuracy_current_run.append(history.history['val_accuracy'][0])
        val_loss_current_run.append(history.history['val_loss'][0])

        test_loss, test_accuracy = genetic_model.evaluate(X_test, y_test, verbose=0)
        test_accuracy_current_run.append(test_accuracy)
        test_loss_current_run.append(test_loss)

    val_accuracy_each_run.append(val_accuracy_current_run)
    val_loss_each_run.append(val_loss_current_run)
    test_accuracy_each_run.append(test_accuracy_current_run)
    test_loss_each_run.append(test_loss_current_run)

average_val_accuracy = np.mean(val_accuracy_each_run, axis=0)
average_val_loss = np.mean(val_loss_each_run, axis=0)
average_test_accuracy = np.mean(test_accuracy_each_run, axis=0)
average_test_loss = np.mean(test_loss_each_run, axis=0)

epochs_range = range(1, epochs + 1)

plt.figure(figsize=(14, 8))
sorted_val_accuracy = np.sort(val_accuracy_each_run, axis=0)
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, sorted_val_accuracy[i], sorted_val_accuracy[i + 1], color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_accuracy, 'r-', label='Average Validation Accuracy', linewidth=3)
plt.title("Validation Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
sorted_test_accuracy = np.sort(test_accuracy_each_run, axis=0)
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, sorted_test_accuracy[i], sorted_test_accuracy[i + 1], color='green', alpha=alpha)
plt.plot(epochs_range, average_test_accuracy, 'r-', label='Average Test Accuracy', linewidth=3)
plt.title("Test Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
sorted_val_loss = np.sort(val_loss_each_run, axis=0)
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, sorted_val_loss[i], sorted_val_loss[i + 1], color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_loss, 'r-', label='Average Validation Loss', linewidth=3)
plt.title("Validation Loss Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
sorted_test_loss = np.sort(test_loss_each_run, axis=0)
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, sorted_test_loss[i], sorted_test_loss[i + 1], color='green', alpha=alpha)
plt.plot(epochs_range, average_test_loss, 'r-', label='Average Test Loss', linewidth=3)
plt.title("Test Loss Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

In [ ]:
## 10 iterations of the image NN ##

import shutil
import os
import random
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.utils import class_weight
import matplotlib.pyplot as plt
from skimage.segmentation import mark_boundaries
from tensorflow.keras.preprocessing import image
import cv2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.resnet50 import preprocess_input, ResNet50


petraea = list(range(1, 8)) + list(range(27,38)) + list(range(42,55)) + [9,23,25,58,59]
print(petraea)
frainetto = list(range(12,15)) + list(range(38,42)) + list(range(55,58)) + [8,19,20,22,24,26]
print(frainetto)
pubescens = list(range(15,19)) + [10,11,21]
print(pubescens)

class_names = ['Frainetto', 'Petraea', 'Pubescens']

input_folder = '/content/Sorted/'
train_folder = 'preprocessed_images/train'
test_folder = 'preprocessed_images/test'

os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

source_dir = '/content/Final_Data_Resized/Final_Data_Resized'

dest_dirs = {
    'petraea': '/content/Sorted/Petraea',
    'frainetto': '/content/Sorted/Frainetto',
    'pubescens': '/content/Sorted/Pubescens'
}

for dir_path in dest_dirs.values():
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

def move_images(numbers, dest_dir):
    for filename in os.listdir(source_dir):
        if filename.endswith(".png"):
            parts = filename.split('_')[0]
            file_number = int(''.join(filter(str.isdigit, parts)))
            if file_number in numbers:
                source_path = os.path.join(source_dir, filename)
                dest_path = os.path.join(dest_dir, filename)
                shutil.move(source_path, dest_path)

move_images(petraea, dest_dirs['petraea'])
move_images(frainetto, dest_dirs['frainetto'])
move_images(pubescens, dest_dirs['pubescens'])


def load_npy_files(folder):
    images = []
    labels = []
    for filename in os.listdir(folder):
        if filename.endswith('.npy'):
            filepath = os.path.join(folder, filename)
            image = np.load(filepath)
            label_part = filename.split('_')[0]

            if label_part == 'Frainetto':
                label = 0
            elif label_part == 'Petraea':
                label = 1
            elif label_part == 'Pubescens':
                label = 2
            images.append(image)
            labels.append(label)
    return np.array(images), np.array(labels)

def preprocess_image(image_path):
    img = Image.open(image_path).convert('RGB')
    img = img.resize((224, 224))
    img = np.array(img).astype(np.float32)
    img = preprocess_input(img)
    return img


image_filenames = []
for class_folder in class_names:
    class_path = os.path.join(input_folder, class_folder)
    for filename in os.listdir(class_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
            image_filenames.append((class_folder, filename))

train_filenames, test_filenames = train_test_split(image_filenames, test_size=0.1, random_state=42)

for class_folder, filename in train_filenames:
    image_path = os.path.join(input_folder, class_folder, filename)
    preprocessed_image = preprocess_image(image_path)

    output_path = os.path.join(train_folder, f"{class_folder}_label_{class_names.index(class_folder)}_{os.path.splitext(filename)[0]}.npy")
    np.save(output_path, preprocessed_image)

for class_folder, filename in test_filenames:
    image_path = os.path.join(input_folder, class_folder, filename)
    preprocessed_image = preprocess_image(image_path)

    output_path = os.path.join(test_folder, f"{class_folder}_label_{class_names.index(class_folder)}_{os.path.splitext(filename)[0]}.npy")
    np.save(output_path, preprocessed_image)

train_images, train_labels = load_npy_files(train_folder)
test_images, test_labels = load_npy_files(test_folder)


print(f"Number of training images: {train_images.shape[0]}")
print(f"Number of test images: {test_images.shape[0]}")

print(f"Shape of train labels: {train_labels.shape}")
print(f"Shape of test labels: {test_labels.shape}")


import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

num_classes = 3
num_runs = 10
epochs = 10
batch_size = 32

val_accuracy_each_run = []
test_accuracy_each_run = []
val_loss_each_run = []
test_loss_each_run = []

for run in range(num_runs):
    resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
    resnet_base.trainable = False

    image_input = Input(shape=(224, 224, 3))
    x = resnet_base(image_input)
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    image_output = Dense(num_classes, activation='softmax')(x)

    image_model = Model(inputs=image_input, outputs=image_output)
    image_model.compile(optimizer=Adam(learning_rate=1e-3), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    val_accuracy_current_run = []
    val_loss_current_run = []
    test_accuracy_current_run = []
    test_loss_current_run = []

    for epoch in range(epochs):
        history = image_model.fit(train_images, train_labels, batch_size=batch_size, epochs=1,
                                  validation_data=(test_images, test_labels), verbose=0)

        val_accuracy_current_run.append(history.history['val_accuracy'][0])
        val_loss_current_run.append(history.history['val_loss'][0])

        test_loss, test_accuracy = image_model.evaluate(test_images, test_labels, verbose=0)
        test_accuracy_current_run.append(test_accuracy)
        test_loss_current_run.append(test_loss)

    val_accuracy_each_run.append(val_accuracy_current_run)
    val_loss_each_run.append(val_loss_current_run)
    test_accuracy_each_run.append(test_accuracy_current_run)
    test_loss_each_run.append(test_loss_current_run)

average_val_accuracy = np.mean(val_accuracy_each_run, axis=0)
average_val_loss = np.mean(val_loss_each_run, axis=0)
average_test_accuracy = np.mean(test_accuracy_each_run, axis=0)
average_test_loss = np.mean(test_loss_each_run, axis=0)

epochs_range = range(1, epochs + 1)


plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, np.sort(val_accuracy_each_run, axis=0)[i],
                     np.sort(val_accuracy_each_run, axis=0)[i + 1], color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_accuracy, 'r-', label='Average Validation Accuracy', linewidth=3)
plt.title("Validation Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, np.sort(test_accuracy_each_run, axis=0)[i],
                     np.sort(test_accuracy_each_run, axis=0)[i + 1], color='green', alpha=alpha)
plt.plot(epochs_range, average_test_accuracy, 'r-', label='Average Test Accuracy', linewidth=3)
plt.title("Test Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, np.sort(val_loss_each_run, axis=0)[i],
                     np.sort(val_loss_each_run, axis=0)[i + 1], color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_loss, 'r-', label='Average Validation Loss', linewidth=3)
plt.title("Validation Loss Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range, np.sort(test_loss_each_run, axis=0)[i],
                     np.sort(test_loss_each_run, axis=0)[i + 1], color='green', alpha=alpha)
plt.plot(epochs_range, average_test_loss, 'r-', label='Average Test Loss', linewidth=3)
plt.title("Test Loss Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
## 10 iterations of the combined NN ##

from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
genetic_data = X
image_data = np.concatenate([train_images, test_images], axis=0)
labels = y

num_runs = 10
epochs = 10
batch_size = 32

val_accuracy_each_run = []
test_accuracy_each_run = []
val_loss_each_run = []
test_loss_each_run = []

for run in range(num_runs):
    print(f"Run {run + 1}/{num_runs}")

    X_train_gen, X_test_gen, y_train, y_test = train_test_split(
        genetic_data, labels, test_size=0.1, random_state=42 + run, stratify=labels
    )
    X_train_img, X_test_img, y_train_img, y_test_img = train_test_split(
        image_data, labels, test_size=0.1, random_state=42 + run, stratify=labels
    )

    scaler = StandardScaler()
    X_train_gen = scaler.fit_transform(X_train_gen)
    X_test_gen = scaler.transform(X_test_gen)

    merged = Concatenate()([image_output, genetic_output])

    x = Dense(12, activation='swish', kernel_regularizer=l2(0.01))(merged)
    x = Dropout(0.4)(x)

    output = Dense(num_classes, activation='softmax')(x)

    combined_model = Model(inputs=[image_input, genetic_input], outputs=output)

    optimizer = Adam(learning_rate=1e-3)
    combined_model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    val_accuracy_current_run = []
    val_loss_current_run = []
    test_accuracy_current_run = []
    test_loss_current_run = []

    for epoch in range(epochs):
        history = combined_model.fit(
            [X_train_img, X_train_gen], y_train,
            batch_size=batch_size,
            epochs=1,
            validation_split=0.3,
            verbose=0
        )

        val_accuracy_current_run.append(history.history['val_accuracy'][0])
        val_loss_current_run.append(history.history['val_loss'][0])

        test_loss, test_accuracy = combined_model.evaluate([X_test_img, X_test_gen], y_test, verbose=0)
        test_accuracy_current_run.append(test_accuracy)
        test_loss_current_run.append(test_loss)

    val_accuracy_each_run.append(val_accuracy_current_run)
    val_loss_each_run.append(val_loss_current_run)
    test_accuracy_each_run.append(test_accuracy_current_run)
    test_loss_each_run.append(test_loss_current_run)

average_val_accuracy = np.mean(val_accuracy_each_run, axis=0)
average_val_loss = np.mean(val_loss_each_run, axis=0)
average_test_accuracy = np.mean(test_accuracy_each_run, axis=0)
average_test_loss = np.mean(test_loss_each_run, axis=0)

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range,
                     np.sort(val_accuracy_each_run, axis=0)[i],
                     np.sort(val_accuracy_each_run, axis=0)[i + 1],
                     color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_accuracy, 'r-', label='Average Validation Accuracy', linewidth=3)
plt.title("Validation Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range,
                     np.sort(test_accuracy_each_run, axis=0)[i],
                     np.sort(test_accuracy_each_run, axis=0)[i + 1],
                     color='green', alpha=alpha)
plt.plot(epochs_range, average_test_accuracy, 'r-', label='Average Test Accuracy', linewidth=3)
plt.title("Test Accuracy Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range,
                     np.sort(val_loss_each_run, axis=0)[i],
                     np.sort(val_loss_each_run, axis=0)[i + 1],
                     color='blue', alpha=alpha)
plt.plot(epochs_range, average_val_loss, 'r-', label='Average Validation Loss', linewidth=3)
plt.title("Validation Loss Ribbon Plot")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()

plt.figure(figsize=(14, 8))
for i in range(num_runs - 1):
    alpha = 0.2 + (i / num_runs) * 0.8
    plt.fill_between(epochs_range,
                     np.sort(test_loss_each_run, axis=0)[i],
                     np.sort(test_loss_each_run, axis=0)[i + 1],
                     color='green', alpha=alpha)
plt.plot(epochs_range, average_test_loss, 'r-', label='Average Test Loss', linewidth=3)
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.grid()
plt.show()
